In [1]:
# !pip install -r requirements.txt
import matplotlib.pyplot as plt
import numpy as np
import time
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

/home/codespace/.python/current/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Section 1 : Load the Model

model_name = "./models/gpt2"

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(model_name)


In [3]:
print(model)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)


In [4]:
# Section 2. How to generate a token from the model output tensors


In [5]:
# 1. Tokenize input prompt

prompt = "The strong ML engineer can do"
inputs = tokenizer(prompt, return_tensors ="pt")
inputs 

{'input_ids': tensor([[  464,  1913, 10373, 11949,   460,   466]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1]])}

In [8]:
# pass the inputs to the model and retrieve the logits 
with torch.no_grad():
    outputs = model(**inputs)

logits = outputs.logits 
print(logits.shape)

torch.Size([1, 6, 50257])


In [9]:
last_logits = logits[0,-1,:]
next_token_id = last_logits.argmax()
next_token_id

tensor(340)

In [10]:
# decode the most likely token 
tokenizer.decode(next_token_id)

' it'

In [11]:
# the 10 next words 
top_k = torch.topk(last_logits, k = 10)
tokens = [tokenizer.decode(tk) for tk in top_k.indices]
tokens

[' it',
 ' a',
 ' this',
 ' the',
 ' anything',
 ' everything',
 ' all',
 ' things',
 ' what',
 ' much']

In [14]:
next_inputs = {
    "input_ids":torch.cat(
        [inputs["input_ids"], next_token_id.reshape((1,1))],
        dim=1
    ),
    "attention_mask": torch.cat(
        [inputs["attention_mask"], torch.tensor([[1]])],
        dim = 1
    )
}

In [15]:
print(next_inputs["input_ids"],
      next_inputs["input_ids"].shape)
print(next_inputs["attention_mask"],
      next_inputs["attention_mask"].shape)

tensor([[  464,  1913, 10373, 11949,   460,   466,   340]]) torch.Size([1, 7])
tensor([[1, 1, 1, 1, 1, 1, 1]]) torch.Size([1, 7])


ok so the idea is that model generates next word, by knowing what it said last time

The input_ids and attention_mask are updated iteratively to reflect the growing sequence, ensuring that the model's predictions are based on all the previous context. This way, the model "remembers" what it has generated so far, which influences its predictions for the next words or tokens in the sequence.



Absolutely, the generation process can't and shouldn't continue indefinitely. There are several strategies used to determine when a model should stop generating further tokens:

Maximum Length: One common approach is to set a maximum sequence length. Once the generated sequence reaches this length, the generation process stops. This prevents excessively long outputs and ensures computational efficiency.
End-of-Sequence Token: Many models are trained to generate a special token (often denoted as <EOS> for End Of Sequence) that signals the completion of a sentence or a logical end to the output. When the model generates this token, the process stops.
Probability Threshold: In some cases, generation can be stopped if the model's confidence in its next predictions falls below a certain threshold. If the predicted probability of the next token is very low, it might indicate that the model is unsure how to continue meaningfully.
Conditional Ending: For specific tasks, the generation might stop based on contextual conditions or rules. For example, in a question-answering task, the model might stop once it has generated what it determines to be a complete answer.
Human Intervention: In interactive applications, a human might decide when the output is sufficient and manually stop the generation.